In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import  pipeline
import torch
import gc

In [25]:
gc.collect()

0

In [26]:
torch.cuda.empty_cache()

In [7]:
SAVE_MODEL_PATH = '/trinity/home/team06/workspace/mikhail_workspace/rag_project/models/vikhr_7B'

In [2]:
model = AutoModelForCausalLM.from_pretrained("Vikhrmodels/Vikhr-7B-instruct_0.4",
                                             device_map="auto",
                                             attn_implementation="flash_attention_2",
                                             torch_dtype=torch.bfloat16)

tokenizer = AutoTokenizer.from_pretrained("Vikhrmodels/Vikhr-7B-instruct_0.4")

Loading checkpoint shards: 100%|██████████| 4/4 [01:50<00:00, 27.66s/it]


In [5]:
model.save_pretrained(SAVE_MODEL_PATH)
tokenizer.save_pretrained(SAVE_MODEL_PATH)

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [23]:



prompts = [
    "Кто такой Михаил Меньщиков?", "Чем известен Михаил Меньщиков?"]

def test_inference(prompt):
    prompt = pipe.tokenizer.apply_chat_template([{'role': 'system', 'content': 'Ты вопросно-ответная система. Отвечай чётко и кратко без лишних подробностей.'},
                                                 {'role': 'assistant', 'content': """Список текстов, на основе которых необходимо генерировать ответ:\n1.Михаил Меньщиков - программный инженер из небольшого городка в Республике Карелия.\n2.Михаил Меньщиков известен своими достижениями области разработки программного обеспечения с интеграцией технологий машинного и глубокого обучения."""},
                                                 {"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    print(prompt)
    outputs = pipe(prompt, max_new_tokens=512, num_beams=5, eos_token_id=79097, early_stopping=True)
    return outputs[0]['generated_text'][len(prompt):].strip()


for prompt in prompts:
    print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(prompt)}")
    print("-"*50)

    prompt:
Кто такой Михаил Меньщиков?
<s><|im_start|>system
Ты вопросно-ответная система. Отвечай чётко и кратко без лишних подробностей.<|im_end|>
<|im_start|>assistant
Список текстов, на основе которых необходимо генерировать ответ:
1.Михаил Меньщиков - программный инженер из небольшого городка в Республике Карелия.
2.Михаил Меньщиков известен своими достижениями области разработки программного обеспечения с интеграцией технологий машинного и глубокого обучения.<|im_end|>
<|im_start|>user
Кто такой Михаил Меньщиков?<|im_end|>
<|im_start|>assistant

    response:
Михаил Меньщиков - инженер-программист из небольшого городка в Республике Карелия. Он известен своими достижениями в области разработки программного обеспечения, особенно в интеграции технологий машинного и глубокого обучения.
--------------------------------------------------
    prompt:
Чем известен Михаил Меньщиков?
<s><|im_start|>system
Ты вопросно-ответная система. Отвечай чётко и кратко без лишних подробностей.<|im_en